In [ ]:
# Mounting my Google Drive and set the cwd on my dataset
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd /content/drive/My Drive/ICIPE/code_ADMMDIPTV/results

Mounted at /content/drive/
/content/drive/My Drive/ICIPE/code_ADMMDIPTV/results


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
from skimage.io import imread, imsave
import numpy as np

In [ ]:
def line_profile_column(u_tr, x, title, n):
  plt.plot(u_tr[:,n])
  plt.plot(x[:,n])
  plt.title(title)
  plt.show()

def line_profile_row(u_tr, x, title, m):
  plt.plot(u_tr[m,:])
  plt.plot(x[m, :])
  plt.title(title)
  plt.show()

In [ ]:
# original image definition
base_path = 'tomo.png'
#base_path = 'faro.png'
# folderset definition
images  = ['barbara'] 
#folders = ['DIP','ADMM_DIPTV','ADMM_DIPWTV']
folders = ['DIP']
#folders = ['starting']
# noise levels names and noise level choice
noise_levels = ['30', '30', '40','50']
nl = noise_levels[0]
# border crop
#m1, m2 = 30, 470
#n1, n2 = 30, 470

# base image loading
#base_image = np.array(imread(base_path), dtype=np.float64)
#print(np.shape(base_image))
#base_image = base_image[:,:,0]
#crop_base_image = base_image[m1:m2,n1:n2]
values= np.zeros((len(folders),2*len(images)), dtype=float)
j=-2;

psnr_list = []
ssim_list = []
for image in images:
  base_path = f'%s.png'%(image)
  base_image = np.array(imread(base_path), dtype=np.float64)
  print(np.shape(base_image))
  i=0
  j=j+2
  for folder in folders:
    actual_path = f'%s/%s_%s_%s.png'%(folder, folder,image,nl)
    #actual_image = np.loadtxt(actual_path)
    actual_image = np.array(imread(actual_path), dtype=np.float64)
    #print(np.shape(actual_image))
    #crop_actual_image=actual_image[m1:m2,n1:n2]
    psnr = compare_psnr(actual_image[:,:,:]/255,base_image[:,:,:]/255)
    ssim = compare_ssim(actual_image[:,:,:]/255, base_image[:,:,:]/255,multichannel=True)
    psnr_list.append(round(psnr, 3))
    ssim_list.append(round(ssim, 3))
    values[i,j]= round(psnr, 3)
    values[i,j+1]= round(ssim, 3)
    i= i+1

    #save_path = f'%s_deblur_%s.png'%(folder, nl)
    #imsave(save_path, actual_image)

    #line_profile_row(base_image, actual_image, folder, 90)

(576, 704, 3)


In [ ]:
values

array([[24.473,  0.794]])

In [ ]:
arrays = [['cameraman','cameraman','faro','faro','hill','hill','I08','I08','qrcode','qrcode','tomo','tomo','I09','I09'], ['PSNR', 'SSIM','PSNR', 'SSIM','PSNR', 'SSIM','PSNR', 'SSIM','PSNR', 'SSIM','PSNR', 'SSIM','PSNR', 'SSIM']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['', ''])

In [ ]:
data = {'PSNR': psnr_list, 'SSIM': ssim_list}
df = pd.DataFrame(values, index=folders,columns=index)
df

In [ ]:
df.to_latex()

'\\begin{tabular}{lrrrrrrrrrrrr}\n\\toprule\n{} & \\multicolumn{2}{l}{cameraman} & \\multicolumn{2}{l}{faro} & \\multicolumn{2}{l}{hill} & \\multicolumn{2}{l}{I08} & \\multicolumn{2}{l}{qrcode} & \\multicolumn{2}{l}{tomo} \\\\\n{} &      PSNR &   SSIM &    PSNR &   SSIM &    PSNR &   SSIM &    PSNR &   SSIM &    PSNR &   SSIM &    PSNR &   SSIM \\\\\n\\midrule\nDIP        &    23.510 &  0.728 &  25.925 &  0.860 &  27.424 &  0.862 &  22.717 &  0.780 &  20.907 &  0.749 &  27.463 &  0.894 \\\\\nDIPTV      &    24.988 &  0.831 &  26.252 &  0.883 &  28.165 &  0.892 &  22.751 &  0.773 &  21.733 &  0.774 &  27.333 &  0.903 \\\\\nADMM\\_DIPTV &    25.583 &  0.846 &  26.474 &  0.891 &  27.549 &  0.884 &  22.850 &  0.782 &  21.938 &  0.784 &  29.996 &  0.915 \\\\\n\\bottomrule\n\\end{tabular}\n'